### Importing the relevant modules and libraries

In [1]:
import pandas as pd
import geopandas as gpd

import folium

import requests
import secret
import json

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

### Reading in the data
<br> Original data retrieved from https://www.ncss.gov.sg/social-service-agencies/membership/list-of-ncss-members
<br> The current data had the mailing address column geocoded

In [2]:
df = pd.read_csv('data/social-service-w-latlong.csv')
df.head()

,S/n,Organisation Name,Mailing Address,Latitude,Longitude,Phone Number,Fax Number,Email Address
0,1,"*SCAPE CO., LTD.",2 Orchard Link #04-01 *SCAPE Building Singapor...,1.301029,103.835732,65216565,65216566,enquiries@scape.sg
1,2,365 Cancer Prevention Society,1 Commonwealth Lane #03-10 ONE Commonwealth Si...,1.304522,103.796819,63373368,NaN,admin@365cps.org.sg
2,3,Abilities Beyond Limitations And Expectations ...,7A Lor 8 Toa Payoh #02-09 Agape Village Singap...,1.340085,103.857026,68017460,NaN,enquiries@able-sg.org
3,4,Action For AIDS (Singapore),9 Kelantan Lane #03-01 Singapore 208628,1.306281,103.856462,62540212,62565903,info@afa.org.sg
4,5,ADULLAM LIFE COUNSELLING,151 Chin Swee Road #08-04 Manhattan House Sing...,1.287301,103.840613,90295323,NaN,admin@adullam.org.sg


### Converting the data to a geopandas dataframe

Using a function, we convert the pandas dataframe to a geopandas dataframe

In [3]:
def create_gdf(df):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Latitude'], df['Longitude']))
    return gdf

# y = long
# x = lat

In [4]:
ss_gdf = create_gdf(df)
ss_gdf.head()


,S/n,Organisation Name,Mailing Address,Latitude,Longitude,Phone Number,Fax Number,Email Address,geometry
0,1,"*SCAPE CO., LTD.",2 Orchard Link #04-01 *SCAPE Building Singapor...,1.301029,103.835732,65216565,65216566,enquiries@scape.sg,POINT (1.301 103.836)
1,2,365 Cancer Prevention Society,1 Commonwealth Lane #03-10 ONE Commonwealth Si...,1.304522,103.796819,63373368,NaN,admin@365cps.org.sg,POINT (1.305 103.797)
2,3,Abilities Beyond Limitations And Expectations ...,7A Lor 8 Toa Payoh #02-09 Agape Village Singap...,1.340085,103.857026,68017460,NaN,enquiries@able-sg.org,POINT (1.340 103.857)
3,4,Action For AIDS (Singapore),9 Kelantan Lane #03-01 Singapore 208628,1.306281,103.856462,62540212,62565903,info@afa.org.sg,POINT (1.306 103.856)
4,5,ADULLAM LIFE COUNSELLING,151 Chin Swee Road #08-04 Manhattan House Sing...,1.287301,103.840613,90295323,NaN,admin@adullam.org.sg,POINT (1.287 103.841)


### Plotting the points on a map using folium for a visualisation

In [5]:
map_folium = folium.Map([1.35255,103.82580], zoom_start=11.4)

point_ss = zip(ss_gdf.Latitude, ss_gdf.Longitude, ss_gdf['Organisation Name'])


for point in point_ss:
    folium.Marker(location=(point[0], point[1]),
                        icon=folium.Icon(color="red"),
                        popup=point[-1]).add_to(map_folium)
    
display(map_folium)

### Geocoding the person's location using Google Maps Geocoding API

In [6]:
input_address = 'Harbourfront MRT station'

In [7]:
url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

querystring = {"address":input_address,"language":"en"}

headers = {
    'x-rapidapi-key': secret.get_secret_key(),
    'x-rapidapi-host': secret.get_secret_host()
    }

response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text)

In [8]:
# function to convert the json text to dictionary
def jprint(obj):
    text = json.loads(obj)
    return text

output = jprint(response.text)

In [9]:
lat_lng = output.get("results")[0].get("geometry").get("location")
coords = lat_lng['lat'], lat_lng['lng']
coords

(1.2653889, 103.8205139)

### Plotting the user's location on the folium map for visualisation

In [10]:
folium.Marker(location=coords,
                        icon=folium.Icon(color="blue"),
                        popup='User').add_to(map_folium)

display(map_folium)

### Nearest neighbour analysis

First, we create a unary union from Points in ss_gdf (which is creating a MultiPoint object from the Point geometrics)

In [11]:
unary_union = ss_gdf.unary_union

Finding the geometry of social service nearest to the user

In [12]:
nearest_geoms = nearest_points(Point(coords), unary_union)

Finding the social service based on matching geometry

In [13]:
matching_geometry = ss_gdf.loc[ss_gdf.geometry == nearest_geoms[1]]  

print(matching_geometry.iloc[0][1])

The Mission to Seafarers Singapore
